In [1]:
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
num_classes = 30  
batch_size = 32

In [3]:
pwd

'C:\\Users\\sayed'

In [3]:
data_dirs = r'C:\Users\sayed\Downloads\Medicinal Leaf Dataset\Medicinal Leaf Dataset\Segmented Medicinal Leaf Images'
data_dir= 'Segmented Medicinal Leaf Images'


In [4]:
# Créez un générateur d'images avec augmentation de données
data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% des données pour la validation
)
input_shape = (224, 224, 3)


In [5]:
# Chargez les données d'entraînement et de validation en utilisant le générateur
train_generator = data_generator.flow_from_directory(
    data_dir,
    target_size = input_shape[:2],
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Utilisez la sous-partie d'entraînement
)

Found 1479 images belonging to 30 classes.


In [6]:
valid_generator = data_generator.flow_from_directory(
    data_dir,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Utilisez la sous-partie de validation
)

Found 356 images belonging to 30 classes.


In [7]:
# Chargement du modèle pré-entraîné InceptionV3 avec les poids ImageNet
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

In [36]:
# Ajout de couches personnalisées au modèle InceptionV3
x = base_model.output
print(x)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)
print(predictions)


AttributeError: 'KerasTensor' object has no attribute 'size'

In [9]:
# Création du modèle final
model = Model(inputs=base_model.input, outputs=predictions)


In [10]:
# Compilation du modèle
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Utilisation de l'arrêt précoce
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [12]:
# Entraînement du modèle
model.fit(
    train_generator,
    epochs=1,
    callbacks=[early_stopping],
    validation_data=valid_generator  # Remplacez par votre générateur de validation
)

47/47 [==============================] - 713s 14s/step - loss: 2.3023 - accuracy: 0.4178 - val_loss: 1.5658 - val_accuracy: 0.5758


### Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model.evaluate(valid_generator)  
print(f"Test InceptionV3 accuracy: {test_accuracy * 100:.2f}%")

In [13]:
test_loss, test_accuracy = model.evaluate(valid_generator)
print(f"Test InceptionV3 accuracy: {test_accuracy * 100:.2f}%")

12/12 [==============================] - 30s 2s/step - loss: 1.5956 - accuracy: 0.5478
Test InceptionV3 accuracy: 54.78%


In [17]:
model.save_weights(r"C:\Users\sayed\Downloads\weights")

In [35]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np
import os

# Load the saved model
model = tf.keras.applications.InceptionV3(
    include_top=False,
    weights=None,  # Weights will be loaded from your saved file
    input_shape=(224, 224, 3)
)
model.load_weights(r"C:\Users\sayed\Downloads\weights")

# Define a function to classify a single image
def classify_image(image_path):
    
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)

    # pred = Dense(num_classes, activation='softmax')(img)
    # Make a prediction
    op = model(img)
    x = GlobalAveragePooling2D()(op)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predctions = Dense(num_classes, activation='softmax')(x)
    
    class_index = np.argmax(x)
    
    return class_index  # This will be the predicted class index

# Define a dictionary to map class indices to class names
class_names = {
    0: "Class1",
    1: "Class2",
    # Add class names for all 30 classes here
}

# Specify the path to the new image you want to classify
new_image_path =r"C:\Users\sayed\Downloads\Medicinal Leaf Dataset\Medicinal Leaf Dataset\Segmented Medicinal Leaf Images\Alpinia Galanga (Rasna)\AG-S-037.jpg"

# Call the classify_image function to get the predicted class index
predicted_class_index = classify_image(new_image_path)
print(predicted_class_index)

# Map the class index to the class name
predicted_class_name = class_names.get(predicted_class_index, "Unknown")

print(f"Predicted class: {predicted_class_name}")


524
Predicted class: Unknown
